In [ ]:
# NOTE: The `ndv` package is not yet supported in Colab. Most of the `ndv` lines are commented out.
# The `matplotlib` package can be used instead (already included in the pip list).

# Manders' Correlation Coefficients

In [ ]:
%pip install matplotlib
# %pip install "ndv[jupyter,vispy]"
%pip install scikit-image
%pip install numpy
%pip install scipy
%pip install tifffile
%pip install imagecodecs
%pip install tqdm
%pip install coloc_tools @ git+https://github.com/fdrgsp/coloc-tools.git

## Description

In this notebook, we will explore how to implement **Manders' Correlation Coefficients** in Python, a common method for quantifying colocalization based on pixel intensities.

The images we will use for this section can be downloaded from the <a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download> <i class="fas fa-download"></i> Manders' & Pearson's Colocalization Dataset</a>.

<p class="alert alert-warning">
   <strong>Note:</strong> This notebook aims to show how to practically implement these methods but does not aim to describe when to use this method. The images used have been selected to showcase the practical implementation of the methods.
</p>

<p class="alert alert-warning">
    <strong>Note:</strong> In this example, we will not perform any image processing steps before computing the Manders' Correlation Coefficients since the image provided has been corrected already. However, when conducting a real colocalization analysis, you should consider applying some image processing steps to clean the images before computing the Manders' Correlation Coefficients, such as background subtraction, flat-field correction, etc. 
</p>

<p class="alert alert-info">
    <strong>Note:</strong> In this notebook, we will only use a single image pair for demonstration purposes. Often, Manders' coefficients should not be interpreted as absolute values in isolation. Instead, it's always recommended to consider them in the context of comparisons between different conditions, controls, treatments, or experimental groups. The relative changes and ratios between conditions are often more meaningful than the absolute coefficient values themselves.
</p>

## Manders' Correlation Coefficients

Manders' correlation coefficients can be used to quantify the degree of colocalization between two channels (or images). These coefficients, M1 and M2, are calculated based on the pixel intensities of the two channels and for this reason, they are different from a simple area overlap.

<div> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/coloc/manders_slide.png" alt="manders" width="800"></div>

**M1** measures the **fraction of channel 1 (**$R^{}$**) intensity that sits where channel 2 (**$G^{}$**) is present**:
- **Numerator**: sum of channel 1 in every pixel (**$R_i^{}$**) where
    - channel 2 is above the channel 2 threshold (**$G_i^{}$**), and
    - channel 1 is above the channel 1 threshold (**$R_i^{}$**)(if you set one).
- **Denominator**: sum of **$R_i^{}$** (channel 1) in every pixel where **$R_i^{}$** is above the **$R_i^{}$**-threshold.

**M2** measures the **fraction of channel 2 (**$G^{}$**) intensity that sits where channel 1 (**$R^{}$**) is present**:
- **Numerator**: sum of channel 2 in every pixel (**$G_i^{}$**) where
	1. channel 1 (**$R_i^{}$**) is above the channel 1 threshold (**$R_i^{}$**), and
	2. channel 2 (**$G_i^{}$**) is above the channel 2 threshold (**$G_i^{}$**) (if you set one).
- **Denominator**: sum of channel 2 in every pixel (**$G_i^{}$**) where channel 2 (**$G_i^{}$**) is above the channel 2 threshold (**$G_i^{}$**).

For this exercise, we will analyze an image of a HeLa cell stained with two fluorescent markers: **channel 1** labels **endosomes** and **channel 2** labels **lysosomes** (<a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download><i class="fas fa-download"></i>Manders' & Pearson's Colocalization Dataset</a>.)

From a biological perspective, lysosomes are typically found within or closely associated with endosomal compartments, while endosomes have a broader cellular distribution. Based on this biology, we expect:

- **High M2 coefficient**: Most lysosomal signal should colocalize with endosomal regions
- **Lower M1 coefficient**: Only a subset of endosomal signal should colocalize with lysosomes, since endosomes are more widely distributed throughout the cell.

### Import Libraries

### Load and Visualize the Image

Open and visualize (with ndv) the image named `cells_manders_14na.tif` from the <a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download><i class="fas fa-download"></i> Manders' & Pearson's Colocalization Dataset</a>. This is a two-channel image where channel 1 has stained endosomes and channel 2 has stained lysosomes. 

To compute Manders' Correlation Coefficients, we need **two separate images** (channels). 

What is the image shape? How do we split the channels?

### Calculate Numerators and Denominators for Manders' Correlation Coefficients

The first and key step is to calculate **$R_i^{}$** and **$G_i^{}$** and thus to select which areas of each channel we want to consider for the colocalization analysis. This means we first need to **threshold each images to select only the pixels we want to consider**.

It is therefore evident that Manders' Correlation Coefficients are **sensitive to thresholding**, the way you decide to threshold your images will have a large impact on the results.

For this example, we will first use a simple Otsu thresholding method and later in the notebook we will explore a more automated way of selecting the threshold.

We can plot the raw data and the masks in a 2x2 subplot to visualize the results of the thresholding.

Now that we have the mask for each channel, we can first **calculate the overlap mask** where both channels are above their respective thresholds, and then calculate **$R_i^{coloc}$** and **$G_i^{coloc}$**.

With the overlap mask, we can now calculate the **$R_i^{coloc}$** (*ch1_coloc*) and **$G_i^{coloc}$** (*ch2_coloc*) and the **numerator** for the Manders' Correlation Coefficients: **sum($R_i^{coloc}$)** and **sum($G_i^{coloc}$)**.

We can now **calculate the denominator** for the Manders coefficients.
<br>
The denominator is the sum of the pixel intensities in the overlap mask for each channel above their respective thresholds: **sum($R_i^{}$)** and **sum($G_i^{}$)**.

### Calculate Manders' Correlation Coefficients

Now with both numerators and denominators calculated, we can compute the Manders coefficients M1 and M2.

With Otsu thresholding for both channels, we obtain:

**M1=0.3496** and **M2=0.8975**

- **M1** indicates that approximately **35%** (0.3496) of channel 1's intensity colocalizes with channel 2. This means that about one-third of channel 1's signal overlaps with areas where channel 2 is also present above threshold.

- **M2** indicates that approximately **90%** (0.8975) of channel 2's intensity colocalizes with channel 1. This suggests that nearly all of channel 2's signal overlaps with areas where channel 1 is also present above threshold.

This asymmetry (M1 ≠ M2) is common and tells us that **channel 2 is largely contained within areas where channel 1 is present**, but **channel 1 extends beyond the regions where channel 2 is found**.

**Bonus**: In the <a href="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/08_pixel_intensity_based_coloc.zip" download> <i class="fas fa-download"></i> Manders' & Pearson's Colocalization Dataset</a> there is an image named `cells_manders_0.3na.tif`, the exact same image we just used nut acquired with a smaller numerical aperture (NA) of the objective lens.

What do you think will happen to the Manders' coefficients if we use this image instead?

### Costes Auto-Threshold Method

As mentioned above, the Mender's Correlation Coefficients are sensitive to thresholding, so the way you decide to threshold your images will have a large impact on the results.

The [`pca_auto_threshold`](https://github.com/fdrgsp/coloc-tools/blob/fee98bb72ccdbffabdc0d4875a9d4fccd43cc8ab/src/coloc_tools/_costes_auto_threshold.py#L796) function from `coloc_tools` implements the [Costes auto-threshold method](https://pmc.ncbi.nlm.nih.gov/articles/PMC1304300/), which **automatically determines optimal threshold values for both channels**. 

The method works by finding threshold values where pixels *below* these thresholds show no statistical correlation (Pearson correlation coefficient ≈ 0). This approach helps objectively separate true signal from background noise.    The algorithm performs orthogonal regression (PCA) between the two channels to establish their relationship, then iteratively tests threshold pairs derived from this regression to identify the optimal separation point between signal and background.

Of course, the Costes auto-threshold method has limitations and may not work in certain scenarios, including:
- **Insufficient data**: When there are too few non-zero pixels (< 10) in either channel
- **No linear relationship**: When channels show non-linear, multiple population, or no correlation patterns.
- **Low variance**: When one or both channels have uniform or near-uniform intensities
- **High background noise**: When noise dominates the signal relationship
- **Limited dynamic range**: Narrow intensity ranges or saturated pixels

In such cases, alternative thresholding methods (Otsu, manual, percentile-based) may be more appropriate.

We can now try to compute and print the Manders' Correlation Coefficients using the Costes auto-threshold method.

**Bonus:** Plot also a scatter plot of the two channels with the linear regression line.

Note that the `fiji_bisection_auto_threshold` function returns (in order) *Costes thresholds for channel 1*, *Costes thresholds for channel 2*, *slope* and *intercept* of the linear regression. We can use the slope and intercept to plot the linear regression line.

Now that we have the Costes thresholds, we can calculate the Manders' Correlation Coefficients using the Costes thresholds as we did for the Otsu thresholds.

How do thresholds and mask images compare to the Otsu thresholds?

### Image Rotation Test

The **image rotation**, in this context, is a statistical method to validate colocalization significance. This method applies **rotations (90°, 180°, 270°) and flips (horizontal and vertical)** to one channel relative to the other, then recalculates the Manders' coefficients.

**Note for Non-Square Images:** When working with non-square images, rotations by 90° and 270° would change the image dimensions (e.g., a 500×512 image becomes 512×500), making direct comparison impossible. To handle this, the function automatically pads non-square images to square dimensions with zeros before applying rotations, ensuring that all transformations maintain the same image size and allow valid statistical comparisons.

The [manders_image_rotation_test](https://github.com/fdrgsp/coloc_tools/blob/8155900fd0140c6a84b79e3c76cf80c9611e5fbc/src/coloc_tools/_manders_image_rotation_test.py#L4) function provides an implementation of this method in Python. This function returns the Manders' coefficients, the rotated/flipped Manders' coefficients, and the p-values for both coefficients.

A low `p-value` (e.g. 0.0001) means that none of the rotations/flips produced M1/M2 values as high as the observed values without translation, indicating that the observed colocalization is statistically significant: the probability of getting the observed colocalization by random chance is < 0.0001 (less than 0.01%).

Let's run it on the two channels we have been working with.

Calculate either the Otsu or Costes thresholds for the two channels and then run the image translation randomization test.

We can now print the results of the analysis: M1, M2, and their respective p-values.

**Bonus:** We can also visualize the distribution of the random M1 and M2 values using a bar plot.

To do this we can simply use the [`manders_image_rotation_test_plot`](https://github.com/fdrgsp/coloc-tools/blob/5a77363e4f6f7fcd8360870b314c3a33fa24a1df/src/coloc_tools/_manders_image_rotation_test.py#L137) function from `coloc_tools`, which takes the M1 and M2 values, the rotated M1 and M2 values, and the p-values for both coefficients as input.

In [ ]:
manders_image_rotation_test_plot(
    M1,
    M2,
    rot_m1_values,
    rot_m2_values,
    p_value_m1_rot,
    p_value_m2_rot,
)

### Image Translation Randomization Test

The **image translation randomization** test is a statistical method used to **validate the significance of colocalization results**, particularly for Manders' coefficients. This method involves **randomly translating one channel relative to another and recalculating the Manders' coefficients** to create a distribution of values under the null hypothesis of no colocalization.

**Note for Image Translations:** Unlike rotations, translations use wraparound shifting (via `np.roll()`) which preserves the original image dimensions regardless of image shape. When pixels are shifted beyond the image boundaries, they wrap around to the opposite side. This ensures that all pixel intensities are preserved and no padding is required, making this method suitable for images of any dimensions.

The [manders_image_translation_randomization](https://github.com/fdrgsp/coloc_tools/blob/2ad1029f1ea5e5e0b0491dbd0b586b3233ce3415/src/coloc_tools/_manders_image_translation_test.py#L5) function from `coloc_tools` provides an implementation of this method in Python. This function returns the Manders' coefficients, the random Manders' coefficients, and the p-values for both coefficients.

A low `p-value` (e.g. 0.0001) means that none of the `n` random translations (by default 1000) produced M1/M2 values as high as the observed values without translation, indicating that the observed colocalization is statistically significant: the probability of getting the observed colocalization by random chance is < 0.0001 (less than 0.01%).

Let's run it on the two channels we have been working with.

Calculate either the Otsu or Costes thresholds for the two channels and then run the image translation randomization test.

We can now print the results of the analysis: M1, M2, and their respective p-values.

**Bonus:** We can also visualize the distribution of the random M1 and M2 values using histograms. This will help us understand the significance of our observed values in the context of the random distributions.

### Summary

The Python implementation for calculating Manders' Correlation Coefficients is straightforward and concise, as demonstrated in the code below.

```python
# Create binary mask for channel 1 & 2 using a thresholding method of choice
threshold_ch1, threshold_ch2 = threshold_method(ch1, ch2)
image_1_mask = ch1 > threshold_ch1
image_2_mask = ch2 > threshold_ch2

# Find pixels that are above threshold in both channels
overlap_mask = image_1_mask & image_2_mask

# Extract channel 1 & 2 intensities only from overlapping regions
ch1_coloc = ch1[overlap_mask]
ch2_coloc = ch2[overlap_mask]

# Extract all channel 1 & 2 intensities above threshold
ch1_tr = ch1[image_1_mask]
ch2_tr = ch2[image_2_mask]

# Calculate total intensity of channel 1 & 2 above threshold
sum_ch1_tr = np.sum(ch1_tr)
sum_ch2_tr = np.sum(ch2_tr)

# M1: fraction of channel 1 intensity that colocalizes with channel 2
M1 = np.sum(ch1_coloc) / sum_ch1_tr
# M2: fraction of channel 2 intensity that colocalizes with channel 1
M2 = np.sum(ch2_coloc) / sum_ch2_tr
```

**Key Considerations for Manders' Correlation Analysis:**

1. **Threshold Sensitivity**: Manders' coefficients are **highly sensitive to thresholding methods**. The choice of thresholding strategy will significantly impact your results, making careful threshold selection essential for accurate colocalization analysis. Consider using:
   - **Automated thresholding methods** (like Costes auto-threshold): these are fully automated and don't require you to select or tune any parameters
   - **Threshold calculation algorithms** (like Otsu, Li, Triangle, Yen, etc.): these automatically calculate a threshold value, but you need to choose which algorithm to use based on your image characteristics and how the resulting threshold looks
   - **Consistent thresholding**: use the same thresholding approach across experimental conditions

2. **Background Considerations**: sometimes it is necessary to apply appropriate image preprocessing steps before calculating Manders' coefficients such as Background subtraction to remove non-specific signal or Flat-field correction to account for illumination variations.

3. **Statistical Validation**: always validate your results using statistical tests such as the image rotation test or the image translation randomization test demonstrated above. This helps assess whether observed colocalization is statistically significant or could have occurred by chance.

4. **Comparative Analysis**: Manders' coefficients should not be interpreted as absolute values in isolation. Instead, consider them in the context of:
   - Comparisons between different experimental conditions
   - Control vs. treatment groups
   - Different time points or developmental stages
   - Relative changes between conditions are often more meaningful than absolute values

5. **Asymmetry Interpretation**: remember that M1 ≠ M2 is common and biologically meaningful.
   - **M1**: Fraction of channel 1 intensity that overlaps with channel 2
   - **M2**: Fraction of channel 2 intensity that overlaps with channel 1
   - This asymmetry can reveal important biological relationships between the labeled structures